In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
with open('train_test_splittedData_pic.pkl', 'rb') as file:
    X_train,X_test,y_train,y_test = pickle.load(file)

In [3]:
num_classes=10

In [4]:
##### convert Everything to ready to train array
X_train = np.array(list(X_train['pic']))
X_test = np.array(list(X_test['pic']))
y_train = np.array(y_train)
y_test = np.array(y_test)

In [5]:
### picture need to be 3d
X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)

#### the shape is >>> a list of 28*28 picture with 1 color channel

In [6]:
## pretty balance __ let validate using this
np.unique(y_train[-1500:],return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([140, 150, 151, 151, 148, 157, 160, 170, 132, 141]))

In [7]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
datagen = ImageDataGenerator(
   rotation_range=15, width_shift_range=0.1,
    height_shift_range=0.1, zoom_range=0.2, fill_mode='nearest')

In [8]:
### let get simple model with dataAugmentation
model = keras.models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               204928    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

In [10]:
checkpointPath = './NNmodel/modelbest.bpMnist_val_br{val_loss:.2f}_{val_acc:.2f}.h5'
cb = [
    keras.callbacks.ModelCheckpoint(checkpointPath,verbose=1,
                                    monitor='val_loss', save_weights_only=True, save_best_only=True,mode='min'),
    keras.callbacks.ReduceLROnPlateau(min_lr=0.0001),
    keras.callbacks.EarlyStopping(patience=20,restore_best_weights=True)
]

In [11]:
model.compile(optimizer='adam',
              loss=keras.losses.categorical_crossentropy,
              metrics=['acc'],
             )

In [12]:
X_tr , x_val, y_tr, y_val = train_test_split(X_train,y_train,shuffle=False,test_size=0.2)

In [13]:
# fits the model on batches with real-time data augmentation:
model.fit(datagen.flow(X_tr, y_tr, batch_size=32),
          steps_per_epoch=len(X_tr) / 32,validation_data=(x_val, y_val), epochs=500,callbacks=cb)


  ...
    to  
  ['...']
Train for 396.8125 steps, validate on 3175 samples
Epoch 1/500
396/396 [============================>.] - ETA: 0s - loss: 2.0191 - acc: 0.4000
Epoch 00001: val_loss improved from inf to 0.33029, saving model to ./NNmodel/modelbest.bpMnist_val_br0.33_0.91.h5
397/396 [==============================] - 4s 10ms/step - loss: 2.0157 - acc: 0.4010 - val_loss: 0.3303 - val_acc: 0.9061
Epoch 2/500
395/396 [============================>.] - ETA: 0s - loss: 0.7690 - acc: 0.7509
Epoch 00002: val_loss improved from 0.33029 to 0.14312, saving model to ./NNmodel/modelbest.bpMnist_val_br0.14_0.96.h5
397/396 [==============================] - 4s 9ms/step - loss: 0.7674 - acc: 0.7514 - val_loss: 0.1431 - val_acc: 0.9587
Epoch 3/500
396/396 [============================>.] - ETA: 0s - loss: 0.4994 - acc: 0.8446
Epoch 00003: val_loss improved from 0.14312 to 0.11078, saving model to ./NNmodel/modelbest.bpMnist_val_br0.11_0.97.h5
397/396 [==============================] - 4s 9ms/st

In [14]:
model.evaluate(X_test,y_test)

3969/3969 [==============================] - 0s 66us/sample - loss: 0.0356 - acc: 0.9907


[0.03557527587412795, 0.9906778]

In [ ]:
###### accuracy 99.07 %
###### multiclass logloss